In [3]:
#Understand the magic in the previus task
dates = ['cica', 'kutya', 'mérésihiba']
dates = [date.upper() for date in dates]

In [4]:
print(dates)

['CICA', 'KUTYA', 'MÉRÉSIHIBA']


# Webscraping multiple pages
https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/

In [7]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

## Collecting all bestseller book titles

In [8]:
#testing the method:
page = 1
while page != 6:
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    print(url)
    page = page + 1

https://www.bookdepository.com/bestsellers?page=1
https://www.bookdepository.com/bestsellers?page=2
https://www.bookdepository.com/bestsellers?page=3
https://www.bookdepository.com/bestsellers?page=4
https://www.bookdepository.com/bestsellers?page=5


In [19]:
page = 1
titles = []
while page != 35:
    
    #basic webscrape procedure
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    
    #get all title:
    for h3 in soup.find_all("h3", class_="title"):
        # .get_text() removes the tags like <h3>
        # strip remove whitespaces
        titles.append(h3.get_text(strip = True))
    page = page + 1

In [21]:
len(titles)

1020

In [22]:
for title in titles[:6]:
    print(title)

It Ends With Us: The most heartbreaking novel you'll ever read
Heaven Official's Blessing: Tian Guan Ci Fu (Novel) Vol. 2
Seven Husbands of Evelyn Hugo
Ugly Love
The Love Hypothesis
Chainsaw Man, Vol. 1


## Get the formats 

In [27]:
#formats = soup.find_all("p", class_="format")s
# the same:
formats = soup.select("div.item-info p.format")

In [28]:
formats[:3]

[<p class="format">Paperback</p>,
 <p class="format">Hardback</p>,
 <p class="format">Hardback</p>]

In [29]:
#So it clears, like .get_text() and create a pandas series
format_series = pd.Series(formats)

In [31]:
format_series[:3]

0    [Paperback]
1     [Hardback]
2     [Hardback]
dtype: object

In [33]:
#get all formats (same method with titles):

page = 1
formats_all = []
while page != 35:
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    
    formats_on_page = soup.select("div.item-info p.format")
    
    #Add that element wich one is on scrape (index)
    for product_format in formats_on_page:
        formats_all.append(product_format.get_text())
        
    page = page + 1
    
formats_series = pd.Series(formats_all)
formats_series.value_counts()

Paperback              799
Hardback               175
Board book              20
Cards                   13
Game                     3
Book                     3
Mixed media product      2
Spiral bound             1
Toy                      1
Novelty book             1
Leather                  1
Sheet music              1
dtype: int64

In [34]:
#Get the percentage value:
formats_series.value_counts(normalize = True)

Paperback              0.783333
Hardback               0.171569
Board book             0.019608
Cards                  0.012745
Game                   0.002941
Book                   0.002941
Mixed media product    0.001961
Spiral bound           0.000980
Toy                    0.000980
Novelty book           0.000980
Leather                0.000980
Sheet music            0.000980
dtype: float64